In [2]:
import os
import numpy as np
import pandas as pd
from congress import Congress
import datetime as DT
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
os.chdir('C:/Users/Asger/Dropbox/Økonomi/Topics in Social Data Science/Projekt/Asger')

### Loading data

In [128]:
df_dem = pd.read_csv('df_dem_final_wZero.csv',delimiter=',', index_col =0)
df_dem = df_dem.drop(['member1_id','member2_id','fullname_1','fullname_2','twitter_account_1','twitter_account_2'], axis=1)

df_rep = pd.read_csv('df_rep_final_wZero.csv',delimiter=',', index_col =0)
df_rep = df_rep.drop(['member1_id','member2_id','fullname_1','fullname_2','twitter_account_1','twitter_account_2'], axis=1)

## Statistic for the democrats

In [86]:
df_dem.head()

,agree_percent,same_gender,same_age,same_state,neighborState,ademic,jaccard
1,96.37,0,0,0,0,5.651718,0.262626
6,93.80,1,1,0,0,8.121424,0.258741
7,92.74,1,1,0,0,3.711670,0.195402
9,94.11,0,0,0,0,2.589895,0.176471
14,93.00,1,0,0,0,5.917988,0.272727


In [87]:
df_dem.describe()

,agree_percent,same_gender,same_age,same_state,neighborState,ademic,jaccard
count,12720.000000,12720.000000,12720.000000,12720.000000,12720.000000,12720.000000,12720.000000
mean,92.959891,0.550000,0.443082,0.075472,0.088915,2.629384,0.118110
std,3.592907,0.497513,0.496769,0.264161,0.284632,3.286322,0.130923
min,75.090000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,91.860000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,93.870000,1.000000,0.000000,0.000000,0.000000,1.093887,0.066667
75%,95.220000,1.000000,1.000000,0.000000,0.000000,4.594020,0.217391
max,98.860000,1.000000,1.000000,1.000000,1.000000,25.317763,0.778571


In [68]:
len(df_dem)

12720

In [69]:
df_dem.corr()
#Very low correlation btw agree_percet and jaccard

,agree_percent,same_gender,same_age,same_state,neighborState,ademic,jaccard
agree_percent,1.000000,-0.027982,0.034306,0.024501,-0.020148,-0.002936,0.013339
same_gender,-0.027982,1.000000,-0.014252,-0.011366,0.013297,-0.031933,-0.009990
same_age,0.034306,-0.014252,1.000000,-0.017590,-0.001182,-0.003637,0.003004
same_state,0.024501,-0.011366,-0.017590,1.000000,-0.089257,-0.116122,-0.124768
neighborState,-0.020148,0.013297,-0.001182,-0.089257,1.000000,-0.023228,-0.021151
ademic,-0.002936,-0.031933,-0.003637,-0.116122,-0.023228,1.000000,0.959355
jaccard,0.013339,-0.009990,0.003004,-0.124768,-0.021151,0.959355,1.000000


### Statistic for the republicans

In [63]:
df_rep.describe()

,agree_percent,same_gender,same_age,same_state,neighborState,ademic,jaccard
count,20555.000000,20555.000000,20555.000000,20555.000000,20555.000000,20555.000000,20555.000000
mean,92.922906,0.839747,0.514133,0.035612,0.108830,2.751687,0.111282
std,4.784585,0.366849,0.499812,0.185325,0.311433,3.546651,0.122500
min,17.650000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,91.740000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,93.820000,1.000000,1.000000,0.000000,0.000000,1.062756,0.069565
75%,95.560000,1.000000,1.000000,0.000000,0.000000,4.560971,0.193277
max,100.000000,1.000000,1.000000,1.000000,1.000000,24.813599,1.000000


In [21]:
len(df_rep)

20555

In [70]:
# Correlation 
df_rep.corr()

#Very low correlation btw agree_percet and jaccard

,agree_percent,same_gender,same_age,same_state,neighborState,ademic,jaccard
agree_percent,1.000000,-0.036055,0.079948,0.045630,0.025542,-0.017240,-0.008938
same_gender,-0.036055,1.000000,0.002270,0.014531,-0.004477,0.018973,0.031946
same_age,0.079948,0.002270,1.000000,0.014000,0.003402,-0.002196,-0.015322
same_state,0.045630,0.014531,0.014000,1.000000,-0.067153,-0.103258,-0.115852
neighborState,0.025542,-0.004477,0.003402,-0.067153,1.000000,-0.002111,-0.003807
ademic,-0.017240,0.018973,-0.002196,-0.103258,-0.002111,1.000000,0.957450
jaccard,-0.008938,0.031946,-0.015322,-0.115852,-0.003807,0.957450,1.000000


# Prediction

In [71]:
df_train_rep = df_rep
df_train_dem = df_dem

df_train_rep.head()

,agree_percent,same_gender,same_age,same_state,neighborState,ademic,jaccard
135,94.76,1,0,0,0,6.627831,0.167568
144,92.42,1,0,0,0,1.514426,0.040462
156,93.95,1,1,0,0,1.014241,0.027778
159,95.64,1,0,0,0,7.734355,0.193370
162,95.64,1,1,0,0,6.576091,0.176136


In [72]:
# Rescaling the data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_train_rep[['agree_percent']] =scaler.fit_transform(df_train_rep[['agree_percent']])
df_train_dem[['agree_percent']] =scaler.fit_transform(df_train_dem[['agree_percent']])

In [73]:
#Defining X and y for dem
X_dem = df_train_dem.columns[1:7]
y_dem = np.array(df_train_dem['agree_percent'])

#Defining X and y for rep
X_rep = df_train_rep.columns[1:7]
y_rep = np.array(df_train_rep['agree_percent'])

## Freature importance with random forrest

In [74]:
clf_rf = RandomForestRegressor(max_depth=5, random_state=0)
clf_rf.fit(df_train_dem[X_dem], y_dem)
print(clf_rf.feature_importances_)

[0.0131111  0.03136779 0.00707113 0.01416319 0.43474546 0.49954134]


In [75]:
clf_rf.fit(df_train_rep[X_rep], y_rep)
print(clf_rf.feature_importances_)

[0.04493555 0.15512942 0.02702691 0.00820097 0.39640524 0.36830192]


## Estimation with OLS

In [91]:
import statsmodels.api as sm
df_rep_var = df_rep.drop(['ademic'], axis=1)
df_dem_var = df_dem.drop(['ademic'], axis=1)
df_rep_var['const']=1 
df_dem_var['const']=1 

#Defining X and y for dem
X_dem = df_dem_var.columns[1:7]
y_dem = np.array(df_dem_var['agree_percent'])

#Defining X and y for rep
X_rep = df_rep_var.columns[1:]
y_rep = np.array(df_rep_var['agree_percent'])

In [94]:
# Results for democrats
results = sm.OLS(y_dem, df_dem_var[X_dem]).fit()
print(results.summary())

# Jaccard not significant :(

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     7.911
Date:                Wed, 23 May 2018   Prob (F-statistic):           1.89e-07
Time:                        15:48:19   Log-Likelihood:                -34297.
No. Observations:               12720   AIC:                         6.861e+04
Df Residuals:                   12714   BIC:                         6.865e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
same_gender      -0.1937      0.064     -3.028

In [95]:
# Results for democrats
results = sm.OLS(y_rep, df_rep_var[X_dem]).fit()
print(results.summary())

# Jaccard not significant :(

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     43.74
Date:                Wed, 23 May 2018   Prob (F-statistic):           5.00e-45
Time:                        15:48:24   Log-Likelihood:                -61234.
No. Observations:               20555   AIC:                         1.225e+05
Df Residuals:                   20549   BIC:                         1.225e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
same_gender      -0.4794      0.091     -5.293

# Prediction with binary variable

In [129]:
df_rep_bi = df_rep
df_dem_bi = df_dem

df_dem_bi['agree_percent'] = np.where(df_dem_bi.agree_percent>93.82,1,0) # Over median
df_rep_bi['agree_percent'] = np.where(df_rep_bi.agree_percent>93.87,1,0) # Over median
df_dem_bi = df_dem_bi.drop(['ademic'], axis=1)
df_rep_bi = df_rep_bi.drop(['ademic'], axis=1)

df_dem_bi.head()

,agree_percent,same_gender,same_age,same_state,neighborState,jaccard
1,1,0,0,0,0,0.262626
6,0,1,1,0,0,0.258741
7,0,1,1,0,0,0.195402
9,1,0,0,0,0,0.176471
14,0,1,0,0,0,0.272727


In [131]:
msk_dem = np.random.rand(len(df_dem_bi)) < 0.8
train_dem = df_dem_bi[msk_dem]
test_dem = df_dem_bi[~msk_dem]
print(len(train_dem), len(test_dem))

msk_rep = np.random.rand(len(df_rep_bi)) < 0.8
train_rep = df_rep_bi[msk_rep]
test_rep = df_rep_bi[~msk_rep]
print(len(train_rep), len(test_rep))

10233 2487
16420 4135


In [132]:
X_vars = df_dem_var.columns[1:7]
X_train_dem = train_dem[X_vars]
y_train_dem = train_dem.agree_percent

X_test_dem = test_dem[X_vars]
y_test_dem = test_dem.agree_percent

X_train_rep = train_rep[X_vars]
y_train_rep = train_rep.agree_percent

X_test_rep = test_rep[X_vars]
y_test_rep = test_rep.agree_percent

In [137]:
# Logistic regression on democrats
from sklearn.linear_model import LogisticRegression #machine learning module - recomended
from sklearn import metrics
clf_lg = LogisticRegression().fit(X_train_dem, y_train_dem)
pred_dem = clf_lg.predict(X_test_dem)

matrix = metrics.confusion_matrix(y_test_dem,pred_dem)
matrix = matrix/matrix.sum(axis=1) # divide by row counts

print("confusion matrix: nltk.SentimentyAnalyzer",'\n', pd.DataFrame(matrix))
print("Accuracy nltk.SentimentyAnalyzer:",metrics.accuracy_score(y_test_dem,pred_dem))

confusion matrix: nltk.SentimentyAnalyzer 
           0         1
0  0.463987  0.494973
1  0.465662  0.569992
Accuracy nltk.SentimentyAnalyzer: 0.5190993164455167


In [138]:
# random forrest on democratss
from sklearn.ensemble import RandomForestClassifier
clf_rf = RandomForestClassifier(max_depth=2, random_state=0).fit(X_train_dem, y_train_dem)
pred_dem = clf_rf.predict(X_test_dem)

matrix = metrics.confusion_matrix(y_test_dem,pred_dem)
matrix = matrix/matrix.sum(axis=1) # divide by row counts

print("confusion matrix: nltk.SentimentyAnalyzer",'\n', pd.DataFrame(matrix))
print("Accuracy nltk.SentimentyAnalyzer:",metrics.accuracy_score(y_test_dem,pred_dem))

confusion matrix: nltk.SentimentyAnalyzer 
           0         1
0  0.633166  0.338747
1  0.596315  0.449343
Accuracy nltk.SentimentyAnalyzer: 0.5375954965822276


In [139]:
# Logistic regression on republicans
from sklearn.linear_model import LogisticRegression #machine learning module - recomended
from sklearn import metrics
clf_lg = LogisticRegression().fit(X_train_rep, y_train_rep)
pred_rep = clf_lg.predict(X_test_rep)

matrix = metrics.confusion_matrix(y_test_rep,pred_rep)
matrix = matrix/matrix.sum(axis=1) # divide by row counts

print("confusion matrix: nltk.SentimentyAnalyzer",'\n', pd.DataFrame(matrix))
print("Accuracy nltk.SentimentyAnalyzer:",metrics.accuracy_score(y_test_rep,pred_rep))

confusion matrix: nltk.SentimentyAnalyzer 
           0         1
0  0.830262  0.168837
1  0.762367  0.241679
Accuracy nltk.SentimentyAnalyzer: 0.5351874244256348


In [144]:
# random forrest on repocratss
from sklearn.ensemble import RandomForestClassifier
clf_rf = RandomForestClassifier(max_depth=5, random_state=0).fit(X_train_rep, y_train_rep)
pred_rep = clf_rf.predict(X_test_rep)

matrix = metrics.confusion_matrix(y_test_rep,pred_rep)
matrix = matrix/matrix.sum(axis=1) # divide by row counts

print("confusion matrix: nltk.SentimentyAnalyzer",'\n', pd.DataFrame(matrix))
print("Accuracy nltk.SentimentyAnalyzer:",metrics.accuracy_score(y_test_rep,pred_rep))

confusion matrix: nltk.SentimentyAnalyzer 
           0         1
0  0.603298  0.394597
1  0.542677  0.460203
Accuracy nltk.SentimentyAnalyzer: 0.5315598548972189
